In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50527, # vocab size
    "context_length": 1024, # context length
    "emb_dim": 768, # Embedding dimension,
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 1, # dropout rate
    "qkv_bias": False # Query-Key-Value bias
}